In [132]:
import pandas as pd
import geopandas as gpd

In [133]:
df = gpd.read_file('combine_dist.shp')
df.shape

(44, 47)

In [134]:
df = df[df['limit'] != 0]

In [135]:
def get_adj_id(x):
    ids = []
    x = dict(x.filter(regex='d_to_'))
    for col, row in x.items():
        if row != 0:
            ids.append(int(col.split('_')[-1])) 
            
    return ids


In [136]:
df['adj_ids'] = df.apply(get_adj_id, axis=1)

In [137]:
#df = df.iloc[:5]

In [138]:
df = df[['ID', 'adj_ids', 'limit', 'geometry']]

In [139]:
df.head()

,ID,adj_ids,limit,geometry
0,1099,[1434],13.835776,"POLYGON ((484361.020 4406619.241, 484376.741 4..."
5,318,"[851, 302]",12.721748,"POLYGON ((484838.779 4407027.272, 484838.799 4..."
6,851,"[318, 302, 153, 53]",12.286532,"POLYGON ((484488.671 4406993.558, 484488.503 4..."
7,1434,"[1099, 1132]",13.871793,"POLYGON ((484280.669 4406714.843, 484280.680 4..."
8,1490,"[1336, 1371, 1587, 1506, 1568]",12.845614,"POLYGON ((483622.847 4406666.645, 483622.847 4..."


In [140]:
count = 0
while count < 3:
    for index, row in df.iterrows():
        adj_geoms = []
        ID = row.ID
        adjacent_IDs = row.adj_ids
        limit = row.limit

        print(f'ID: {ID}')
        print(f'adjacent_ID: {adjacent_IDs}')
        print(f'limit: {limit}')

        for adj_id in adjacent_IDs:
            adj_geom = df.loc[df['ID'] == int(adj_id), 'geometry'].values[0]
            adj_geoms.append(adj_geom)
       
        new_geoms = [row.geometry] + adj_geoms
        new_df = gpd.GeoDataFrame(geometry=new_geoms, crs=df.crs.to_epsg())
        new_df['ID'] = new_df.index

        print('__________________________')
        
        count += 1

ID: 1099
adjacent_ID: [1434]
limit: 13.835775979150291
__________________________
ID: 318
adjacent_ID: [851, 302]
limit: 12.721748357462596
__________________________
ID: 851
adjacent_ID: [318, 302, 153, 53]
limit: 12.28653156412822
__________________________
ID: 1434
adjacent_ID: [1099, 1132]
limit: 13.871792794578942
__________________________
ID: 1490
adjacent_ID: [1336, 1371, 1587, 1506, 1568]
limit: 12.84561418789523
__________________________
ID: 1336
adjacent_ID: [1490, 1657, 1659]
limit: 13.172089249887028
__________________________
ID: 1132
adjacent_ID: [1434]
limit: 13.871792794578942
__________________________
ID: 377
adjacent_ID: [823]
limit: 12.271765636556795
__________________________
ID: 302
adjacent_ID: [318, 851]
limit: 12.721748357462596
__________________________
ID: 153
adjacent_ID: [851, 53]
limit: 12.620083559738319
__________________________
ID: 53
adjacent_ID: [851, 153, 73, 799]
limit: 13.174123765515343
__________________________
ID: 823
adjacent_ID: [377, 60

In [ ]:
adj_geoms

[<GeometryArray>
 []
 Length: 0, dtype: geometry,
 <GeometryArray>
 []
 Length: 0, dtype: geometry,
 <GeometryArray>
 []
 Length: 0, dtype: geometry,
 <GeometryArray>
 []
 Length: 0, dtype: geometry,
 <GeometryArray>
 []
 Length: 0, dtype: geometry]

In [ ]:
remove_ids = []
tessellations = gpd.GeoDataFrame()
for index, row in merge.iterrows():
    ID = row.ID
    adjacent_ID = row.adjacent_ID[0]
    limit = row.limit
    geometry = row.geometry
    adj_geometry = merge[merge['ID'] == int(adjacent_ID)].geometry.values[0]
    df = gpd.GeoDataFrame(geometry=[geometry, adj_geometry], crs=merge.crs.to_epsg())
    df['ID'] = df.index

    tess = build_tessellation(df, 'ID', limit)
    w = build_spatial_topology(tess, 'rook', 'ID')
    combine = get_components(tess, w)

    sel_parcels = parcels.sjoin(combine, how='inner', op='intersects')
    sel_parcels = sel_parcels[['ID_left', 'geometry']].rename(columns={'ID_left': 'ID'})
    remove_ids.append(sel_parcels.ID.to_list())
    sel_tess = build_tessellation(sel_parcels, 'ID', (limit + 5))
    
    tessellations = pd.concat([tessellations, sel_tess], ignore_index=True)


    
    